In [33]:
!pip install spicy
!pip install catboost
from catboost import CatBoostRegressor, CatBoostClassifier
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, ShuffleSplit, GridSearchCV
import spicy
from spicy import stats
from sklearn.neighbors import LocalOutlierFactor

In [34]:
df = pd.read_csv("/content/dielectron.csv")

In [35]:
df

,Run,Event,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2,M
0,147115,366639895,58.71410,-7.311320,10.531000,-57.29740,12.82020,-2.202670,2.177660,1,11.28360,-1.032340,-1.88066,-11.077800,2.14537,-2.344030,-2.072810,-1,8.94841
1,147115,366704169,6.61188,-4.152130,-0.579855,-5.11278,4.19242,-1.028420,-3.002840,-1,17.14920,-11.713500,5.04474,11.464700,12.75360,0.808077,2.734920,1,15.89300
2,147115,367112316,25.54190,-11.480900,2.041680,22.72460,11.66100,1.420480,2.965600,1,15.82030,-1.472800,2.25895,-15.588800,2.69667,-2.455080,2.148570,1,38.38770
3,147115,366952149,65.39590,7.512140,11.887100,63.86620,14.06190,2.218380,1.007210,1,25.12730,4.087860,2.59641,24.656300,4.84272,2.330210,0.565865,-1,3.72862
4,147115,366523212,61.45040,2.952840,-14.622700,-59.61210,14.91790,-2.093750,-1.371540,-1,13.88710,-0.277757,-2.42560,-13.670800,2.44145,-2.423700,-1.684810,-1,2.74718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,146511,522575834,12.31310,-10.658000,5.164440,3.36858,11.84330,0.280727,2.690370,-1,1.80181,0.668609,-1.58437,0.537805,1.71967,0.307851,-1.171470,1,8.44779
99996,146511,522786431,18.46420,7.854990,15.133000,-7.08659,17.05020,-0.404510,1.092010,1,14.69110,-1.418020,-2.28117,-14.443500,2.68598,-2.383880,-2.126960,1,20.71540
99997,146511,522906124,4.18566,-3.273500,-0.308507,-2.59013,3.28801,-0.723075,-3.047630,1,72.81740,-11.074900,-9.28179,-71.369300,14.45010,-2.300410,-2.444050,-1,12.71350
99998,146511,523243830,54.46220,11.352600,11.880900,51.92400,16.43280,1.867800,0.808132,-1,8.58671,0.378009,3.07828,8.007050,3.10141,1.677170,1.448610,1,4.69670


In [36]:
df.rename(columns={'px1 ':'px1'}, inplace=True)

In [37]:
df.describe()

,Run,Event,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2,M
count,100000.000000,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,99915.000000
mean,147935.327230,4.957212e+08,36.436466,0.135897,0.182291,-1.508037,14.412167,-0.064095,0.021614,-0.00548,44.002901,-0.003984,0.124654,-1.590559,13.802036,-0.072830,0.021385,-0.004220,30.019521
std,920.306064,4.037703e+08,41.216203,13.404976,13.470281,51.603652,12.388740,1.462137,1.799562,0.99999,46.751132,13.127404,13.168867,61.430040,12.460549,1.738033,1.817031,0.999996,25.255847
min,146511.000000,6.414000e+03,0.377928,-250.587000,-126.079000,-840.987000,0.219629,-4.165380,-3.141580,-1.00000,0.472500,-233.730000,-145.651000,-655.396000,0.026651,-7.064790,-3.141580,-1.000000,2.000080
25%,147114.000000,2.023970e+08,8.458595,-5.233675,-5.276248,-15.859825,3.771172,-1.283883,-1.527030,-1.00000,11.055725,-4.794770,-4.605965,-22.036950,3.738103,-1.892287,-1.556925,-1.000000,12.445200
50%,147929.000000,3.902759e+08,21.717000,0.141339,0.099092,-0.312987,12.967800,-0.061178,0.034324,-1.00000,25.264600,-0.035638,0.083665,-0.690244,11.690950,-0.135911,0.026986,-1.000000,21.283100
75%,149011.000000,6.338891e+08,50.003350,5.714560,5.648087,13.212650,20.018925,1.144408,1.562355,1.00000,66.925525,4.819540,5.061860,19.005825,19.596400,1.768253,1.596737,1.000000,39.025050
max,149182.000000,1.791247e+09,850.602000,134.539000,147.467000,760.096000,265.578000,2.622970,3.141420,1.00000,948.375000,227.330000,166.283000,935.558000,281.654000,3.060550,3.141290,1.000000,109.999000


In [38]:
df.isna().sum() #checking null values

Run       0
Event     0
E1        0
px1       0
py1       0
pz1       0
pt1       0
eta1      0
phi1      0
Q1        0
E2        0
px2       0
py2       0
pz2       0
pt2       0
eta2      0
phi2      0
Q2        0
M        85
dtype: int64

In [39]:
df = df.drop(["Run","Event"],axis=1) #as Run and Event number doesn't contribute much and work as index we drop them

In [40]:
data = df.copy()

In [43]:
print(data.head())

         E1       px1        py1       pz1       pt1     eta1     phi1  Q1  \
0  58.71410  -7.31132  10.531000 -57.29740  12.82020 -2.20267  2.17766   1   
1   6.61188  -4.15213  -0.579855  -5.11278   4.19242 -1.02842 -3.00284  -1   
2  25.54190 -11.48090   2.041680  22.72460  11.66100  1.42048  2.96560   1   
3  65.39590   7.51214  11.887100  63.86620  14.06190  2.21838  1.00721   1   
4  61.45040   2.95284 -14.622700 -59.61210  14.91790 -2.09375 -1.37154  -1   

        E2        px2      py2      pz2       pt2      eta2      phi2  Q2  \
0  11.2836  -1.032340 -1.88066 -11.0778   2.14537 -2.344030 -2.072810  -1   
1  17.1492 -11.713500  5.04474  11.4647  12.75360  0.808077  2.734920   1   
2  15.8203  -1.472800  2.25895 -15.5888   2.69667 -2.455080  2.148570   1   
3  25.1273   4.087860  2.59641  24.6563   4.84272  2.330210  0.565865  -1   
4  13.8871  -0.277757 -2.42560 -13.6708   2.44145 -2.423700 -1.684810  -1   

          M  
0   8.94841  
1  15.89300  
2  38.38770  
3   3.72862 

In [44]:
print(data.describe().T)

         count       mean        std         min        25%        50%  \
E1    100000.0  36.436466  41.216203    0.377928   8.458595  21.717000   
px1   100000.0   0.135897  13.404976 -250.587000  -5.233675   0.141339   
py1   100000.0   0.182291  13.470281 -126.079000  -5.276248   0.099092   
pz1   100000.0  -1.508037  51.603652 -840.987000 -15.859825  -0.312987   
pt1   100000.0  14.412167  12.388740    0.219629   3.771172  12.967800   
eta1  100000.0  -0.064095   1.462137   -4.165380  -1.283883  -0.061178   
phi1  100000.0   0.021614   1.799562   -3.141580  -1.527030   0.034324   
Q1    100000.0  -0.005480   0.999990   -1.000000  -1.000000  -1.000000   
E2    100000.0  44.002901  46.751132    0.472500  11.055725  25.264600   
px2   100000.0  -0.003984  13.127404 -233.730000  -4.794770  -0.035638   
py2   100000.0   0.124654  13.168867 -145.651000  -4.605965   0.083665   
pz2   100000.0  -1.590559  61.430040 -655.396000 -22.036950  -0.690244   
pt2   100000.0  13.802036  12.460549  

In [45]:
data["M"].fillna(data["M"].median(),inplace=True). #replacing null values with median
print(data.isnull().sum())

E1      0
px1     0
py1     0
pz1     0
pt1     0
eta1    0
phi1    0
Q1      0
E2      0
px2     0
py2     0
pz2     0
pt2     0
eta2    0
phi2    0
Q2      0
M       0
dtype: int64


In [46]:
DataForOutlier = data.copy()

In [47]:
clf = LocalOutlierFactor()
clf.fit_predict(DataForOutlier)

array([1, 1, 1, ..., 1, 1, 1])

In [48]:
score = clf.negative_outlier_factor_ #checking outliers

In [49]:
scoresorted = np.sort(score)
print(scoresorted[0:20])

[-5.0169149  -4.81691687 -3.26828395 -3.15470092 -2.74668789 -2.68953241
 -2.67087267 -2.64501824 -2.64466029 -2.60517136 -2.55004921 -2.50167581
 -2.44673163 -2.43563456 -2.4174773  -2.40922905 -2.39142776 -2.36714336
 -2.34817962 -2.32834396]


In [50]:
point = scoresorted[4]
print(f"Outlier Point -- > {point} ")
print("---"*20)
print("Outlier Row -- >\n",DataForOutlier[score==point])

Outlier Point -- > -2.7466878910494286 
------------------------------------------------------------
Outlier Row -- >
             E1     px1      py1     pz1      pt1      eta1     phi1  Q1  \
70776  111.216 -65.296 -62.0951  65.189  90.1076  0.671778 -2.38132   1   

            E2      px2      py2      pz2      pt2      eta2     phi2  Q2  \
70776  97.6194 -60.0806 -32.2262  69.8666  68.1777  0.898782 -2.64926  -1   

             M  
70776  27.5021  


In [51]:
totaloutlier = DataForOutlier < point
print(DataForOutlier[totaloutlier].any())

E1      False
px1      True
py1      True
pz1      True
pt1     False
eta1     True
phi1     True
Q1      False
E2      False
px2      True
py2      True
pz2      True
pt2     False
eta2     True
phi2     True
Q2      False
M       False
dtype: bool


In [52]:
print(data[score < point])

              E1        px1         py1       pz1        pt1     eta1  \
18749  850.60200   53.77760  115.639000 -840.9870  127.53200 -2.58506   
21447  838.83000 -250.58700  -87.965200 -795.6790  265.57800 -1.81719   
21789    3.40368   -1.87945   -0.143547    2.8341    1.88492  1.19674   
44885  770.58300  -22.68970  124.649000  760.0960  126.69700  2.49167   

          phi1  Q1         E2        px2        py2        pz2        pt2  \
18749  1.13550   1    2.90757   -2.08829  -1.129120    1.67872    2.37400   
21447 -2.80399  -1   39.72040  -11.83550   4.618670  -37.63370   12.70480   
21789 -3.06536  -1  948.37500  138.74500  69.961800  935.55800  155.38600   
44885  1.75085  -1   18.39200   -4.32788   0.947612   17.85040    4.43041   

           eta2      phi2  Q2        M  
18749  0.658496 -2.645930   1  90.8602  
21447 -1.806410  2.769530   1  40.3669  
21789  2.495210  0.467041  -1  41.1658  
44885  2.101740  2.926040  -1  27.8643  


In [53]:
CleanData = DataForOutlier > point
print(data[CleanData])

             E1        px1        py1       pz1       pt1      eta1      phi1  \
0      58.71410        NaN  10.531000       NaN  12.82020 -2.202670  2.177660   
1       6.61188        NaN  -0.579855       NaN   4.19242 -1.028420       NaN   
2      25.54190        NaN   2.041680  22.72460  11.66100  1.420480  2.965600   
3      65.39590   7.512140  11.887100  63.86620  14.06190  2.218380  1.007210   
4      61.45040   2.952840        NaN       NaN  14.91790 -2.093750 -1.371540   
...         ...        ...        ...       ...       ...       ...       ...   
99995  12.31310        NaN   5.164440   3.36858  11.84330  0.280727  2.690370   
99996  18.46420   7.854990  15.133000       NaN  17.05020 -0.404510  1.092010   
99997   4.18566        NaN  -0.308507  -2.59013   3.28801 -0.723075       NaN   
99998  54.46220  11.352600  11.880900  51.92400  16.43280  1.867800  0.808132   
99999   7.64000   0.886162   5.478900       NaN   5.55010 -0.842662  1.410440   

       Q1        E2        

In [54]:
for i in data.columns: #to determine if a data is normally distributed
    print("---"*30)
    print(i)
    print("%.4f - %.4f" % spicy.stats.shapiro(data[i]))

------------------------------------------------------------------------------------------
E1
0.7573 - 0.0000
------------------------------------------------------------------------------------------
px1
0.9469 - 0.0000
------------------------------------------------------------------------------------------
py1
0.9485 - 0.0000
------------------------------------------------------------------------------------------
pz1
0.8804 - 0.0000
------------------------------------------------------------------------------------------
pt1
0.8553 - 0.0000
------------------------------------------------------------------------------------------
eta1
0.9562 - 0.0000
------------------------------------------------------------------------------------------
phi1
0.9566 - 0.0000
------------------------------------------------------------------------------------------
Q1
0.6374 - 0.0000
------------------------------------------------------------------------------------------
E2
0.7920 - 0.0000
--

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


0.8900 - 0.0000
------------------------------------------------------------------------------------------
phi2
0.9538 - 0.0000
------------------------------------------------------------------------------------------
Q2
0.6367 - 0.0000
------------------------------------------------------------------------------------------
M
0.8421 - 0.0000


In [55]:
print("%.3f - %.3f" % spicy.stats.levene(data["E1"],data["E2"])) #to determine equality of variances assuming a non-normal population
print("%.3f - %.3f" % spicy.stats.levene(data["px1"],data["px2"]))
print("%.3f - %.3f" % spicy.stats.levene(data["py1"],data["py2"]))
print("%.3f - %.3f" % spicy.stats.levene(data["pt1"],data["pt2"]))
print("%.3f - %.3f" % spicy.stats.levene(data["pz1"],data["pz2"]))
print("%.3f - %.3f" % spicy.stats.levene(data["eta1"],data["eta2"]))
print("%.3f - %.3f" % spicy.stats.levene(data["phi1"],data["phi1"]))
print("%.3f - %.3f" % spicy.stats.levene(data["Q1"],data["Q2"]))

1170.060 - 0.000
81.542 - 0.000
76.191 - 0.000
0.503 - 0.478
2091.670 - 0.000
9383.707 - 0.000
0.000 - 1.000
0.079 - 0.778


In [56]:
data.to_csv("Preprocessed_data.csv", index=False)